In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 10]


import sys
sys.path.append('../../')

In [ ]:
from keras.preprocessing import image
import numpy as np

from src.classifier import TLClassifier
from src.utilities import load_dataset_from_path, UIUC_EVENT_CLASS, CODALAB_GENDER_CLASS, CODALAB_SMILE_CLASS, KAGGLE_DOGCAT_CLASS
from src.plots import plot_confusion_matrix

# Codalab Gender

In [ ]:
# Codalab Gender 3 classes
# VGG16 bottleneck path
vgg16_model = TLClassifier('vgg16', 3, 'codalab/224_224/model/gender/vgg16/checkpoint/weights-improvement-189-0.82.hdf5')
inception_model = TLClassifier('inceptionv3', 3, 'codalab/224_224/model/gender/inceptionv3/checkpoint/weights-improvement-132-0.82.hdf5')
resnet50_model = TLClassifier('resnet50', 3, 'codalab/224_224/model/gender/resnet50/checkpoint/weights-improvement-014-0.82.hdf5')

In [ ]:
# Example image
codalab_1_path = '../data/codalab/im_GenFex_06173.jpg'
codalab_1_img = image.load_img(codalab_1_path, target_size=(224, 224))
codalab_1 = image.img_to_array(codalab_1_img)
codalab_1 = np.expand_dims(codalab_1, axis=0)
codalab_1_img

In [ ]:
from sklearn.metrics.classification import accuracy_score, confusion_matrix

print(vgg16_model.predict(codalab_1))
print(inception_model.predict(codalab_1))
print(resnet50_model.predict(codalab_1))

## Predicting with 3 models

In [ ]:
val_data_path = 'codalab/224_224/val_data.npz'
val_label_path = 'codalab/224_224/val_gender_label.npz'
val_data = load_dataset_from_path(val_data_path)[:20]
val_label = load_dataset_from_path(val_label_path)[:20]
metrics = [accuracy_score, confusion_matrix]

print(val_data[0])
vgg16_preds = vgg16_model.predict(val_data)
vgg16_y = np.argmax(vgg16_preds, axis=1)
vgg16_scores = vgg16_model.evaluate_metrics(vgg16_y, val_label, metrics)
print(val_data[0])
inception_preds = inception_model.predict(val_data)
inception_y = np.argmax(inception_preds, axis=1)
inception_scores = inception_model.evaluate_metrics(inception_y, val_label, metrics)
print(val_data[0])
resnet50_preds = resnet50_model.predict(val_data)
resnet50_y = np.argmax(resnet50_preds, axis=1)
resnet50_scores = resnet50_model.evaluate_metrics(resnet50_y, val_label, metrics)
print(val_data[0])

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = [5, 5]
plot_confusion_matrix(plt, vgg16_scores[1], CODALAB_GENDER_CLASS.values())
plot_confusion_matrix(plt, inception_scores[1], CODALAB_GENDER_CLASS.values())
plot_confusion_matrix(plt, resnet50_scores[1], CODALAB_GENDER_CLASS.values())

In [ ]:
# Wrong classified but with high confidence
vgg16_wrong_idx = np.where(vgg16_y != val_label)
vgg16_wrong_idx
np.max(vgg16_preds[vgg16_wrong_idx], axis=1)

resnet50_wrong_idx = np.where(resnet50_y != val_label)
resnet50_wrong_idx_probs = np.max(resnet50_preds[resnet50_wrong_idx], axis=1)

In [ ]:
resnet50_wrong_idx_probs_sorted = np.argsort(resnet50_wrong_idx_probs)[::-1]
resnet50_wrong_idx_probs_sorted

In [ ]:
resnet50_wrong_idx[0]

In [ ]:
np.max(resnet50_preds[resnet50_wrong_idx], axis=1)

In [ ]:
sorted_wrong_idx = resnet50_wrong_idx[0][resnet50_wrong_idx_probs_sorted]

In [ ]:
val_data[sorted_wrong_idx][0]